### Importar as Bibliotecas para rodar o código python OpenAI

In [2]:
#import OpenAI API and other necessary libraries
import os
from dotenv import load_dotenv
from scraper import fetch_website_contents
import requests
from IPython.display import display, Markdown
from openai import OpenAI



### Carregar o envorinment 

In [4]:
# Load environment variables
load_dotenv(override=True)
openai_api_key = os.getenv("OPENAI_API_KEY")

if not openai_api_key:
    display(Markdown("**Error:** OpenAI API key is missing. Please set the `OPENAI_API_KEY` environment variable."))

### Create Prompts for stocks

In [8]:
# Define system and user prompts for stock market analysis
system_prompt = """You are an expert stock market analyst with deep knowledge of financial markets, 
technical analysis, and fundamental analysis. Your role is to provide clear, data-driven insights 
about stocks, market trends, and investment opportunities. You analyze financial data objectively 
and present balanced perspectives on both risks and opportunities."""

user_prompt = """Please analyze the current stock market conditions and provide insights on the following:

1. Overall market sentiment and key trends
2. Company  performance 
3. Notable stock movements and the reasons behind them
4. Potential opportunities and risks for investors
5. Key technical indicators (graphs like moving averages, RSI, MACD and others) to watch for this company as better investment decisions

Please provide a comprehensive analysis based on recent market data."""

print("System Prompt:")
print(system_prompt)
print("\n" + "="*80 + "\n")
print("User Prompt:")
print(user_prompt)

System Prompt:
You are an expert stock market analyst with deep knowledge of financial markets, 
technical analysis, and fundamental analysis. Your role is to provide clear, data-driven insights 
about stocks, market trends, and investment opportunities. You analyze financial data objectively 
and present balanced perspectives on both risks and opportunities.


User Prompt:
Please analyze the current stock market conditions and provide insights on the following:

1. Overall market sentiment and key trends
2. Company  performance 
3. Notable stock movements and the reasons behind them
4. Potential opportunities and risks for investors
5. Key technical indicators (graphs like moving averages, RSI, MACD and others) to watch for this company as better investment decisions

Please provide a comprehensive analysis based on recent market data.


In [ ]:
# Fetch website contents and enhance user prompt with scraped data
url = "https://www.dadosdemercado.com.br/acoes/itub4"  # You can change this to any financial website

def marketData(site):
    site = fetch_website_contents(url)
    # Enhance user prompt with scraped market data
    # Initialize OpenAI client and get response with market data
    client = OpenAI(api_key=openai_api_key)

    # Enhance user prompt with scraped market data
    enhanced_prompt = f"""
    Based on the following real-time market data from {url}:

    {marketData}

    {user_prompt}
    """

    response = client.chat.completions.create(
        model="gpt-5-nano",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": enhanced_prompt}
        ]
    )

    # Display the response
    display(Markdown(response.choices[0].message.content))

    print(f"Successfully scraped content from: {url}")
    print(f"Content length: {len(marketData)} characters")

    print(f"Successfully scraped content from: {url}")
    print(f"Content length: {len(scraped_content)} characters")

In [ ]:
# Initialize OpenAI client and get response
client = OpenAI(api_key=openai_api_key)

response = client.chat.completions.create(
	model="gpt-5-nano",
	messages=[
		{"role": "system", "content": system_prompt},
		{"role": "user", "content": user_prompt}
	]
)

# Display the response
display(Markdown(response.choices[0].message.content))